In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab10.ipynb")

<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2022<br>
            Dr. Eric Van Dusen <br>
        Alan Liang <br>
            Umar Maniku<br>
            Yiyang Chen <br>
            Bennett Somerville <br>
</table>

# Lab 10: Macroeconomic Policy

In [1]:
!pip install fredapi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datascience import * 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.optimize import curve_fit
import math
from math import *
import seaborn as sns
import itertools
from fredapi import Fred
%matplotlib inline

## Macroeconomic Data and FRED API

We will start by first downloading some macroeconomic data from FRED. In this notebook, we will introduce the FRED API (Application Programming Interface).  

**What is an API?**  
In contrast to a user interface, which connects a computer to a person, an application programming interface (API) connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them. [Wikipedia]

### Part 1: Get the API Key

In most cases, you will need to get an API key in order to access an API. For some resources, it involves some paperwork to apply and/or limited free usage. For FRED, the process of getting an API key is simple. Make an account and request the API key [here](https://fred.stlouisfed.org/docs/api/api_key.html).

**Question 1.1:** Request a FRED API key and paste it below.

<!--
BEGIN QUESTION
name: q1_1
-->

In [2]:
my_api_key = "YOUR API KEY HERE"


In [ ]:
grader.check("q1_1")

### Part 2: Access the API using the `fredapi` package

In lecture, we discussed how to use APIs generally. In this lab, we skip straight to using the `fredapi` package to interact with the FRED API. First, we initialize a `Fred` object with our API key.

In [4]:
fred = Fred(api_key=my_api_key)

To access data from the FRED API, we need to know the series ID for the data we want. You can find this using the [FRED](https://fred.stlouisfed.org) website. For example, by searching for GDP, we can see on the [FRED page for Gross Domestic Product](https://fred.stlouisfed.org/series/GDP) that the series ID is GDP, as written in parentheses.

In [5]:
gdp_df = pd.DataFrame(fred.get_series('GDP'))
gdp_df.head(10)

In [6]:
gdp_df.plot(figsize=(15, 10));

<!-- BEGIN QUESTION -->

**Question 2.1**: Find the series ID for the federal funds rate, and use it to graph the federal funds rate over time.

*Hint: There are multiple series ID for the federal funds rate; use the one that updates monthly. You may need to click "2 other formats" to find the series that updates monthly.*

<!--
BEGIN QUESTION
name: q2_1
manual: true
-->

In [7]:
fred.get_series(...).plot(figsize=(15, 10)); 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.2**: How does the FFR over the past 15 years compare to the FFR before then, and how is this related to the discussion of the FFR in lecture? What may have caused the initial decrease? 2-3 sentences will suffice.

<!--
BEGIN QUESTION
name: q2_2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.3**: Find the series ID for the unemployment rate, and use it to graph the unemployment rate over time.

<!--
BEGIN QUESTION
name: q2_3
manual: true
-->

In [8]:
fred.get_series(...).plot(figsize=(15, 10));

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.4**: What spikes do you see over the past 30 years? What do they correlate with? 2-3 sentences will suffice.

<!--
BEGIN QUESTION
name: q2_4
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.5**: Find the series ID for the consumer price index that excludes food and energy, and use it to graph inflation over time. In this case, we want to get quarterly data and put the data in units of percent change from a year ago, because the percent change in CPI tells us the inflation rate. To find the values of these parameters parameters, look at [the FRED documentation for the `fred/series/observations` endpoint](https://fred.stlouisfed.org/docs/api/fred/series_observations.html).


<!--
BEGIN QUESTION
name: q2_5
manual: true
-->

In [9]:
fred.get_series(..., frequency=..., units=...).plot(figsize=(15, 10));

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.6**: What do you notice about inflation over the past 2 years? In 2008? Why might these trends exist? 2-3 sentences will suffice.

<!--
BEGIN QUESTION
name: q2_6
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Part 3: Use the FRED API to graph a Phillips curve

<!-- BEGIN QUESTION -->

**Question 3.1**: Recalling what you learned in lecutre, what macroeconomic indicators does the Phillips curve relate? Is the correlation usually negative or positive? What is the intuition behind this relationship? 3-4 sentences will suffice.

<!--
BEGIN QUESTION
name: q3_1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



To start, we will use the unemployment rate and inflation data to graph a Phillips curve. The `fred` package's `get_series` function returns a Pandas series, which we turn into a Pandas dataframe. To convert to Data 8 tables, we use, the `datascience` package's `Table.from_df` function.

In [10]:
unemployment_table = Table.from_df(
    pd.DataFrame(
        fred.get_series('UNRATE')
    ).rename(columns={0:'unemployment_rate'}).reset_index()
)
unemployment_table

Next, we will grab the inflation data like we did in question 2.5.

In [11]:
inflation_table = Table.from_df(
    pd.DataFrame(
        fred.get_series('CPILFESL', units='pc1', frequency='q')
    ).rename(columns={0:'inflation_rate'}).dropna().reset_index()
)
inflation_table

<!-- BEGIN QUESTION -->

**Question 3.2**: We now have to *join* the two datasets. This should be familiar from Data 8, but if you need a quick refresher, feel free to reference over [section 8.4 in the Data 8 textbook](https://inferentialthinking.com/chapters/08/4/Joining_Tables_by_Columns.html). Each row in our `phillips_curve_table` should report the unemployment rate and CPI for the same date, if `unemployment_table` and `inflation_table` both have data at that date.

<!--
BEGIN QUESTION
name: q3_2
manual: true
-->

In [24]:
phillips_curve_table = ...
phillips_curve_table

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 3.3**: Graph the Phillips curve, showing the relationship between the unemployment rate and inflation.

<!--
BEGIN QUESTION
name: q3_3
manual: true
-->

In [15]:
phillips_curve_table.scatter(..., ..., width=15, height=10)
plt.xlabel("Unemployment")
plt.ylabel("Inflation");

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Question 3.4**: Let's add a "decade" column to the table so we can color the points based on what decade they fell in. To do this, write a function (named `find_year_for_date`) that returns the decade of a given date, and then apply it to the `index` column on the `phillips_curve_table`. If you need a refresher on the `apply` function, consult [the Data 8 textbook](https://inferentialthinking.com/chapters/08/1/Applying_a_Function_to_a_Column.html).

*Hint: you can get the year of the `date` parameter with `date.year`.*

<!--
BEGIN QUESTION
name: q3_4
manual: true
-->

In [16]:
def find_year_for_date(date):
    the_year = ...
    nearest_decade = ...
    return nearest_decade

phillips_curve_table = phillips_curve_table.with_columns(
    'decade', 
    ...
)
phillips_curve_table

<!-- END QUESTION -->



The code for the graph below is beyond the scope of this course. You don't need to understand it but feel free to find the documentation for the seaborn package if you're interested.

In [20]:
plt.figure(figsize=(15, 10))
sns.scatterplot(x="unemployment_rate", y="inflation_rate", data=phillips_curve_table, 
                hue="decade", palette="bright", alpha=0.75)
plt.xlabel("Unemployment")
plt.ylabel("Core Inflation")
plt.show();

We can add best fit lines for each decade, too.

In [21]:
sns.lmplot(x="unemployment_rate", y="inflation_rate", data=phillips_curve_table.to_df(), 
                hue="decade", palette="bright", ci=None, height=10)
plt.xlabel("Unemployment")
plt.ylabel("Core Inflation")
plt.show();

<!-- BEGIN QUESTION -->

**Question 3.5**: What trends do you notice? Recalling question 3.1, does the traditional relationship behind the Phillips curve always hold? If not, why might that be?

<!--
BEGIN QUESTION
name: q3_5
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



**Congratulations!** You are done with your final lab Data 88E this semester! This lab gave you a brief taste of applying data science to macroeconomic indicators. We sincerely hope you enjoyed this lab, as well as our course in general. Please remember to fill out your course evaluations so we can learn from your feedback! 

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)